<img src="NotebookAddons/blackboard-banner.png" width="100%" />

# InSAR Time Series Analysis using MintPy and HyP3 products

**Author:** Alex Lewandowski; University of Alaska Fairbanks

Based on the [LosAngeles_time_series](https://github.com/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Hazards/LosAngeles_time_series.ipynb) notebook by Eric Fielding, David Bekaert, Heresh Fattahi and Yunjun Zhang, which uses an example ARIA dataset.



## Mapping surface deformation with InSAR time series

This notebook demonstrates how to create an InSAR time series with MintPy using an ASF HyP3 InSAR data stack, which can be ordered on [ASF Data Search/Vertex](https://search.asf.alaska.edu/#/) and prepared using the [Prepare_HyP3_InSAR_Stack_for_MintPy](https://github.com/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb) notebook.

<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />

It also explores how to assess the quality of the stack inversion, temporal coherence, and velocity errors.

**Important Note about JupyterHub**
Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

# 0: Notebook setup

**Import necessary packages**

In [ ]:
import contextlib
import csv
from datetime import datetime
import h5py
import os
from pathlib import Path
import re
from tqdm.notebook import tqdm
import urllib
import zipfile

from ipyfilechooser import FileChooser

import numpy as np
from osgeo import gdal, osr

try:
    import rasterio
    from rasterio.transform import from_origin
except:
    !mamba install -c conda-forge rasterio --yes
    import rasterio
    from rasterio.transform import from_origin

import mintpy.view
import mintpy.tsview
import mintpy.plot_network
import mintpy.plot_transection
import mintpy.plot_coherence_matrix
import mintpy.objects.insar_vs_gps
import mintpy.utils

import opensarlab_lib as asfn

**Select the directory holding your MintPy-ready HyP3 data stack**
- Click the `Select` button
- Navigate to your data directory
- Click the `Select` button
- Confirm that the desired path appears in green text
- Click the `Change` button to alter your selection

In [ ]:
fc = FileChooser('/home/jovyan/notebooks')
display(fc)

**Define a project name and create a MintPy directory in which to store files output during our analysis**

In [ ]:
# define the work directory
work_path = Path(fc.selected_path)
print(f"Work directory: {work_path}")

# define a project name
proj_name = input("Enter a project name: ")

# define the MintPy time-series directory
mint_path = work_path/'MintPy'
if not mint_path.is_dir():
    mint_path.mkdir()
print(f"MintPy directory: {mint_path}")

#create a directory in which to store plots
plot_path = mint_path/"plots"
if not plot_path.is_dir():
    plot_path.mkdir()

# 1. Add Your Climate Data Store (CDS) UID & API Key to the Pyaps3 Config 

### This step only needs to be completed once and may be skipped if you have already updated the config for your conda environment

- Running the MintPy smallbaselineApp's `correct_troposphere` step requires downloading atmospheric pressure data from the CDS
- If don't yet have a CDS API key:
    - Proceed to [CDS](https://cds.climate.copernicus.eu/cdsapp#!/home) and create an account
    - Open the [Datasets page](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset)
    - Search for "ERA5"
    - Select any of ERA5 datasets that appear
    - Select the `Download data` tab
    - Scroll to the bottom of the screen
    - Accept the `Terms of use`
    - Click on your name at the top right of the screen to access your profile page
    - Your `UID` and `API Key` will be displayed here 
- Run the following 2 code cells to update the pyaps config from this notebook **OR** open an OpenSARlab terminal and complete the following steps:
    - Use vim or another text editor to open the `model.cfg` config file in your `insar_analysis` conda environment's `pyaps3` directory
        - If you built the `insar_analysis` environment using the Create_OSL_Conda_Environments.ipynb notebook, it will be located here:
            - `~/.local/envs/insar_analysis/lib/python3.8/site-packages/pyaps3/model.cfg`
        - Add your `UID` and `API Key` to the `[CDS]` profile in the config
            - This should be formatted like:
                - key = `<UID>:<API Key>`
                    - *note: do not include less-than and greater-than characters*
        - Save the config and exit the text editor

**If you do not add your CDS credentials to `model.cfg`, the `correct_troposphere` step will fail**

In [ ]:
from getpass import getpass

pyaps_cfg = Path("/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/pyaps3/model.cfg")
with open(pyaps_cfg, 'r') as f:
    lines = f.readlines()
    if "your-uid" in lines[5] or "your-api-key" in lines[5]:
        print("There was no CDS UID or API Key in the pyaps3 model.cfg")
        update_cds_cfg = None
    else:
        print("There was a CDS UID and API Key found in the pyaps3 model.cfg")
        print("Would you like to update them?")
        update_cds_cfg = asfn.select_parameter(["Do not update CDS UID and API Key", 
                                                "Update CDS UID and API Key"])
        display(update_cds_cfg)

In [ ]:
if not update_cds_cfg or "Update" in update_cds_cfg.value:
    with open(pyaps_cfg, 'w') as f:
            uid = input("Enter your CDS UID")
            key = getpass("Enter your CDS API Key")
            lines[5] = f"key = {uid}:{key}\n"
            f.seek(0)
            f.writelines(lines)
            f.truncate()

# 2. smallbaselineApp.py overview

This application provides a workflow which includes several steps to invert a stack of unwrapped interferograms and apply different corrections to obtain ground displacement timeseries.  
The workflow consists of two main blocks:

* correcting unwrapping errors and inverting for the raw phase time-series (blue ovals),
* correcting for noise from different sources to obtain the displacement time-series (green ovals).

Some steps are optional, which are switched off by default (marked by dashed boundaries). Configuration parameters for each step are initiated with default values in a customizable text file: [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg). In this notebook, we will walk through some of these steps, for a complete example see the [MintPy repository](https://github.com/insarlab/MintPy).

<p align="left">
  <img width="600" src="../Hazards/NotebookAddons/MintPyWorkflow.jpg">
</p>     
<p style="text-align: center;">
    (Figure from Yunjun et al., 2019)
</p>

## 2.1. Processing steps of smallbaselineApp.py

The MintPy **smallbaselineApp.py** application provides a workflow to invert a stack of unwrapped interferograms and apply different (often optional) corrections to obtain ground displacement timeseries. A detailed overview of the options can be retrieved by involking the help option:

In [ ]:
!smallbaselineApp.py --help

## 2.2. Configuring processing parameters

The processing parameters for the **smallbaselineApp.py** are controlled through a configuration file. If no file is provided the default [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg) configuration is used. We will create a custom config file with modified configuration parameters for this time-series analysis.

In [ ]:
config = f'''# vim: set filetype=cfg:
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {work_path}/*/*unw_phase_clip.tif
mintpy.load.corFile          = {work_path}/*/*corr_clip.tif
##---------geometry datasets:
mintpy.load.demFile          = {work_path}/*/*dem_clip.tif
mintpy.load.incAngleFile     = {work_path}/*/*lv_theta_clip.tif
mintpy.load.azAngleFile      = {work_path}/*/*lv_phi_clip.tif
mintpy.load.waterMaskFile    = {work_path}/*/*water_mask_clip.tif
'''

In [ ]:
ref_point_option = asfn.select_parameter(["Define a reference point",
                                         "Allow MintPy to determine a reference point"])
display(ref_point_option)

In [ ]:
is_float = False
while not is_float:
    if 'Define' in ref_point_option.value:
        try:
            lat = float(input("Enter reference latitude"))
            lon = float(input("Enter reference longitude"))
            is_float = True
        except ValueError:
            print("Latitude and Longitude must be convertable to float")
            continue
        config = f'{config}\nmintpy.reference.lalo        = {lat},{lon}'
    else:
        break

In [ ]:
ref_date_option = asfn.select_parameter(["Reference time-series to earliest date in stack",
                                         "Allow MintPy to determine reference date"])
display(ref_date_option)

In [ ]:
if 'MintPy' in ref_date_option.value:
    config = f'{config}\nmintpy.reference.date       = auto'
else:
    config = f'{config}\nmintpy.reference.date       = no'

In [ ]:
config_path = mint_path/f'{proj_name}.txt'

with open(config_path, 'w') as f:
    f.write(config)
    print(f"config file path: {config_path}\n")
    
with open(config_path, 'r') as f:
    for line in f.readlines():
        print(line)

# 3. Small Baseline Time Series Analysis

**You can run every step in smallbaselineApp.py with one call, using the command in the cell below**

**For the purposes of this tutorial, we will run each step separately**

We will run the steps:
- load_data
- modify_network
- reference_point
- quick_overview
- invert_network
- correct_troposphere
- correct_topography
- residual_RMS
- reference_date
- velocity
- google_earth

Skipped steps include:
- correct_unwrap_error
- correct_LOD
- correct_SET
- deramp
- hdfeos5

Skipped steps will also be skipped if running the entire smallbaselineApp in the cell below.

In [ ]:
# This runs every step

# !smallbaselineApp.py --work-dir {mint_path}  {config_path}

## 3.1. Load Data

**Run the `load_data` step**

- If you get a missing 'Height' attribute error, you are missing a DEM, which is an available option when ordering HyP3 InSAR products


In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep load_data

The output of the loading step is an "inputs" directory containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata.  
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.). 

In [ ]:
inputs_path = mint_path/'inputs'
!ls $inputs_path

<div class="alert alert-info">
<b>info.py :</b> 
To get general infomation about a MintPy product, run info.py on the file.   
</div>

## 3.2. Modify the Network

**Run the `modify_network` step**

- Identifies and excludes interferograms (i.e. affected by remaining coherence phase-unwrapping errors) before the network inversion

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep modify_network

## 3.3. Plot the interferogram network

Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- ifgramStack_coherence_spatialAvg.txt: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- Network.pdf: Displays the network of interferograms on time-baseline coordinates, colorcoded by avergae coherence of the interferograms. 
- CoherenceMatrix.pdf shows the avergae coherence pairs between all available pairs in the stack.

In [ ]:
%matplotlib inline
with asfn.work_dir(mint_path):
    mintpy.plot_network.main([f'{inputs_path}/ifgramStack.h5'])
    plots = ['bperpHistory.pdf', 'coherenceHistory.pdf', 'coherenceMatrix.pdf', 'network.pdf']
    for p in plots:
        if (mint_path/p).exists():
            (mint_path/p).rename(f'{plot_path}/{p}')
            

## 3.4. Set the Reference Point

**Run the `reference_point` step**

The interferometric phase is a relative observation by nature. The phases of each unwrapped interferogram are relative with respect to an arbitrary pixel. Therfore, we need to reference all interferograms to a common reference pixel.

The `reference_point` step selects a common reference pixel for the stack of interferograms. The default approach of MintPy is to choose a pixel with the highest spatial coherence in the stack. Other options include specifying the longitude and latitude of the desired reference pixel or the line and column number of the refence pixel.   

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_point

**Running the "reference_step" adds additional attributes "REF_X, REF_Y" and "REF_LON, REF_LAT" to the ifgramStack.h5 file. To see the attributes of the file run info.py**

In [ ]:
!info.py $inputs_path/ifgramStack.h5 | egrep 'REF_'

## 3.5. Run a Quick Overview

**Run the `quick_overview` step**

- Assess possible groud deformation using the velocity from traditional interferogram stacking 
    - *reference: Zebker et al. (1997, JGR)*
- Assess distribution of phase unwrapping error from the number of interferogram triplets with non-zero integer ambiguity of closure phase 
    - *reference: T_int in Yunjun et al. (2019, CAGEO). Related to section 3.2, equation (8-9) and Fig. 3d-e.*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep quick_overview

## 3.6. Inverting the Small Baseline network

**Run the `invert_network` step**

- Invert the network of differential unwrapped interferograms to estimate the time-series of unwrapped phase with respect to a reference acquisition date
- By default mintpy selects the first acquisition
- The estimated time-series is converted to distance change from radar to target and is provided in meters. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep invert_network

## 3.7. Correct for Tropospheric Propagation Delays (Optional)

**Run the `correct_troposphere` step**

- Uses ECMWF [ERA5 climate reanalysis pressure data](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&keywords=((%20%22Product%20type:%20Reanalysis%22%20)%20AND%20(%20%22Provider:%20Copernicus%20C3S%22%20))&text=pressure)
- CDS limits ECMWF archive requests to 50, so your requests may be queued until there is space.
    - https://cds.climate.copernicus.eu/live/queue

In [ ]:
tropo_choice = asfn.select_parameter(["Perform Tropospheric Correction Step",
                                      "Skip Tropospheric Correction Step",
                                      "Delete Outputs of a Previous (possibly interrupted) Troposheric Correction and Rerun",
                                      "Delete Outputs of a Previous Troposheric Correction and Skip Troposheric Correction Now"])

display(tropo_choice)

In [ ]:
def set_troposhperic_correction_mintpy(config_path, method):
    config_path = Path(config_path)
    with open(config_path, 'r') as f:
        config = f.readlines()    
    config_update = config
    
    for i, l in enumerate(config):
        no_comment = l.split("#")[0]
        present = False
        if "mintpy.troposphericDelay.method=" in "".join(no_comment.split()):
            config_update[i] = f"mintpy.troposphericDelay.method = {method}"
            present = True
            break     
    if not present:
        config_update.append(f"\nmintpy.troposphericDelay.method = {method}")

    config_str = ""
    for l in config_update:
        config_str = f"{config_str}{l}"
    
    with open(config_path, 'w') as f:
        f.write(config_str)

In [ ]:
correct_tropo = "Perform" in tropo_choice.value or "Rerun" in tropo_choice.value
era5_path = mint_path/"ERA5"
timeseries_era5_path = mint_path/"timeseries_ERA5.h5"
inputs_era5_path = mint_path/"inputs/ERA5.h5"

if "Delete" in tropo_choice.value:
    for f in [timeseries_era5_path, inputs_era5_path]:
        try:
            f.unlink()
        except FileNotFoundError:
            pass
    try:
        shutil.rmtree(era5_path)
    except FileNotFoundError:
        pass

if correct_tropo:
    set_troposhperic_correction_mintpy(config_path, "pyaps")
    !smallbaselineApp.py $config_path --work-dir {mint_path} --dostep load_data
    !smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_troposphere
else:
    set_troposhperic_correction_mintpy(config_path, "no")
    !smallbaselineApp.py $config_path --work-dir {mint_path} --dostep load_data

## 3.8. Correct for DEM Errors

**Run the `correct_topography` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_topography

## 3.9. Calculate the Root Mean Square (RMS) of Residual Phase Time-Series for Each Acquisition

**Run the `residual_RMS` step**

- *reference: Yunjun et al. (2019, section 4.9 and 5.4)*
- To remove the long wavelength component in space, a phase ramp is removed for each acquisition
- Sets optimal reference date to date with min RMS
- Sets exclude dates (outliers) to dates with RMS > cutoff * median RMS (Median Absolute Deviation)

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep residual_RMS

## 3.10. Reference the Entire Time-Series to One Date in Time

**Run the `reference_date` step**

- *reference: Yunjun et al. (2019, section 4.9)*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_date

## 3.11. Estimate The Long-Term Velocity Rate

**Run the `velocity` step**

The timeseries file contains three datasets:
- the `time-series` dataset, which is the interferometric range change for each acquisition relative to the reference acquisition
- the `date` dataset, which contains the acquisition date for each acquisition
- the `bperp` dataset, which contains the timeseries of the perpendicular baseline 

The ground deformation caused by many geophysical or anthropogenic processes are linear at first order approximation. Therefore it is common to estimate the rate of the ground deformation which is the slope of linear fit to the time-series. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep velocity

In [ ]:
scp_args = f'{mint_path}/velocity.h5 velocity -v -1 1 --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity.png'
mintpy.view.main(scp_args.split())

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

## 3.12. Geocode velocity.h5 in Preparation for Creating a velocity.kmz

**Run the `geocode` step**

- This is unnecessary for geocoded HyP3 data but would be needed for non-geocoded data

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep geocode

## 3.13. Create a kmz File

**Run the `google_earth` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep google_earth

## 3.14 View and save GeoTiffs of wrapped and unwrapped inverted timeseries

**Plot the unwrapped inverted time series steps (n, n+1, n+2, etc...)**

- save a png

In [ ]:
if correct_tropo:
    ts_demErr = f'{mint_path}/timeseries_ERA5_demErr.h5'
else:
    ts_demErr = f'{mint_path}/timeseries_demErr.h5'

scp_args = f'{ts_demErr} --notitle --notick --noaxis --dpi 600 --figsize 15 15 --outfile {plot_path}/unwrapped_inverted_ts.png'
mintpy.view.main(scp_args.split())

**Plot the wrapped inverted time series steps (n, n+1, n+2, etc...)**

- save a png

In [ ]:
scp_args = f'{ts_demErr} --wrap --wrap-range -2.8 2.8 --notitle --notick --noaxis --dpi 600 --figsize 15 15 --outfile {plot_path}/wrapped_inverted_ts.png'
mintpy.view.main(scp_args.split())

**Create a list dates for all timesteps**

In [ ]:
ifgramstack = inputs_path/"ifgramStack.h5"

with h5py.File(ifgramstack, "r") as f:
    dates = f["date"][()]
    dates = list(set([d.decode("utf-8") for insar in dates for d in insar]))
    dates.sort()

**Save wrapped interferograms showing total deformation for each each step (n, n+1, n+2, etc...)**

- save GeoTiffs

In [ ]:
with h5py.File(ifgramstack, 'r') as f:
    unw_pth = f.attrs['FILE_PATH']

ds_unw = rasterio.open(unw_pth, 'r', driver='GTiff')

for i, d in enumerate(tqdm(dates)):
    cmd = f'view.py {ts_demErr} {dates[0]}_{dates[i]} --wrap --wrap-range -2.8 2.8 --notitle --notick --noaxis'
    data, atr, inps = mintpy.view.prep_slice(cmd)
    
    print(f'{np.mean(data)}')

    with rasterio.open(f'{plot_path}/wrapped_{dates[0]}_{dates[i]}.tif', 'w', driver='GTiff',
                      height = data.shape[0], width = data.shape[1],
                      count=1, dtype=str(data.dtype),
                      crs=ds_unw.read_crs(),
                      transform=ds.transform) as ds:
        ds.write(data.astype(rasterio.float32), 1)

**Save unwrapped interferograms showing total deformation for each each step (n, n+1, n+2, etc...)**

- Save Geotiffs

In [ ]:
for i, d in enumerate(tqdm(dates)):
    cmd = f'view.py {ts_demErr} {dates[0]}_{dates[i]} --notitle --notick --noaxis'
    data, atr, inps = mintpy.view.prep_slice(cmd)

    rasterio.open(f'{plot_path}/unwrapped_{dates[0]}_{dates[i]}.tif', 'w', driver='GTiff',
                  height = data.shape[0], width = data.shape[1],
                  count=1, dtype=str(data.dtype),
                  crs=ds.read_crs(),
                  transform=ds.transform)   

# 4. Error analysis (signal vs noise)

Uncertainty of the ground displacement products derived from InSAR time-series, depends on the quality of the inversion of the stack of interferograms and the accuracy in separating the ground displacement from other components of the InSAR data. Therefore the definition of signal vs noise is different at the two main steps in MintPy:  

1) During the inversion: 
    At this step all systematic components of the interferometric phase (e.g., ground displacement, propagation delay, geometrical residuals caused by DEM or platform's orbit inaccuracy) are considered signal, while the interferometric phase decorrelation, phase unwrapping error and phase inconsistency are considered noise. 
    
2) After inversion: the ground displacement component of the time-serieses is signal, and everything else (including the propagation delay and geometrical residuals) are considered noise

Therefore we first discuss the possible sources of error during the inversion and the existing ways in MintPy to evaluate the quality of inversion and to improve the uncertainty of the inversion. Afterwards we explain the different components of the time-series and the different processing steps in MintPy to separate them from ground displacement signal.  


## 4.1. Quality of the inversion

The main sources of noise during the time-series inversion includes decorrelation, phase unwrapping error and the inconsistency of triplets of interferofgrams. Here we mainly focus on the decorrelation and unwrapping errors. We first show the existing quantities in MintPy to evaluate decorrelation and unwrapping errors and then discuss the existing ways in MintPy to reduce the decorrelation and unwrapping errors on the time-series inversion.

### 4.1.1. Average spatial coherence

Mintpy computes temporal average of spatial coherence of the entire stack as a potential ancillary measure to choose reliable pixels after time-series inversion.

In [ ]:
%matplotlib inline
scp_args = f"{mint_path}/avgSpatialCoh.h5 --dpi 600 --figsize 15 15 --outfile {plot_path}/avg_spatial_coh.png"
mintpy.view.main(scp_args.split())

### 4.1.2. Temporal coherence

In addition to timeseries.h5 which contains the time-series dataset, invert_network produces other quantities, which contain metrics to evaluate the quality of the inversion including temporalCoherence.h5. Temporal coherence represents the consistency of the timeseries with the network of interferograms. 

Temporal coherence varies from 0 to 1. Pixels with values closer to 1 are considered reliable and pixels with values closer to zero are considered unreliable. For a dense network of interferograms, a threshold of 0.7 may be used (Yunjun et al, 2019).

In [ ]:
%matplotlib inline
scp_args = f"{mint_path}/temporalCoherence.h5 --dpi 600 --figsize 15 15 --outfile {plot_path}/temporal_coh.png"
mintpy.view.main(scp_args.split())

## 4.2. Velocity error analysis

The estimated velocity also comes with an expression of unecrtainty which is simply based on the goodness of fit while fitting a linear model to the time-series. This quantity is saved in "velocity.h5" under the velocityStd dataset. 

In [ ]:
%matplotlib inline
scp_args = f'{mint_path}/velocity.h5 velocityStd -v -0.2 0.2 --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity_err.png'
mintpy.view.main(scp_args.split())

Note that the plot above is the velocity error, not the velocity. The errors generally increases with distance from the reference point and can also increase for points with elevations different from the reference point if topographically correlated water vapor variations are especially strong in the area.

## 4.3. Compare InSAR time-series with GPS time-series in LOS direction

- http://geodesy.unr.edu/NGLStationPages/gpsnetmap/GPSNetMap.html
- http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt

### 4.3.1. Identify Potential GPS stations

**Write the University of Nevada, Reno GPS station holdings metadata to GPS_stations.csv**

In [ ]:
with asfn.work_dir(mint_path):
    url = 'http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt'
    response = urllib.request.urlopen(url, timeout=5)
    content = response.read()
    rows = content.decode('utf-8').splitlines()
    holdings_txt = Path('.')/'DataHoldings.txt'
    if holdings_txt.exists():
        holdings_txt.unlink()

with open(f'{mint_path}/GPS_stations.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', escapechar=',', quoting=csv.QUOTE_NONE)
    for row in rows:
        csv_writer.writerow([re.sub('\s+', ' ', row)])

**Build a list of GPS stations within your area of interest and select one**

In [ ]:
# get the InSAR stack's corner coordinates
with h5py.File(f"{mint_path}/inputs/geometryGeo.h5", 'r') as f:
    lon_west = float(f.attrs['LON_REF1'])
    lon_east = float(f.attrs['LON_REF2'])
    lat_south = float(f.attrs['LAT_REF1'])
    lat_north = float(f.attrs['LAT_REF3'])

# get the start and end dates of the timeseries
if correct_tropo:
    info = gdal.Info(f"{mint_path}/timeseries_ERA5_demErr.h5", format='json')
else:
    info = gdal.Info(f"{mint_path}/timeseries_demErr.h5", format='json')
ts_start = info['metadata']['']['START_DATE']
ts_start = datetime.strptime(ts_start, '%Y%m%d')
ts_end = info['metadata']['']['END_DATE']
ts_end = datetime.strptime(ts_end, '%Y%m%d')

# find all stations that have data within the ts time range,
# are located within the AOI and at an unmasked pixel location
gps_stations = list()
with open(f'{mint_path}/GPS_stations.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in list(csv_reader)[1:]:
        begin_date = datetime.strptime(row[7], '%Y-%m-%d')
        mod_date = datetime.strptime(row[9], '%Y-%m-%d')
        lat = float(row[1])
        if float(row[2]) > 180:
            lon = float(row[2]) - 360
        else:
            lon = float(row[2])

        n = [lat, lon]
        a = [lat_north, lon_west]
        b = [lat_south, lon_west]
        c = [lat_south, lon_east]
        ab = np.subtract(a, b)
        an = np.subtract(a, n)
        bc = np.subtract(b, c)
        bn = np.subtract(b, n)

        in_aoi = 0 <= np.dot(ab, an) <= np.dot(ab, ab) and 0 <= np.dot(bc, bn) <= np.dot(bc, bc)
        in_date_range = ts_start >= begin_date and ts_end <= mod_date
        
        if in_aoi and in_date_range:
            vel_file = f'{mint_path}/velocity.h5'
            atr = mintpy.utils.readfile.read_attribute(vel_file)
            coord = mintpy.utils.utils.coordinate(atr, lookup_file=f'{mint_path}/inputs/geometryRadar.h5')
            y, x = coord.geo2radar(lat, lon)[:2]
            msk = mintpy.utils.readfile.read(f'{mint_path}/maskTempCoh.h5')[0]
            box = (x, y, x+1, y+1)
            masked = not msk[y, x]
            if not masked:
                gps_stations.append(row[0].strip())
                
if len(gps_stations) > 0:
    gps_station = asfn.select_parameter(gps_stations)
    print("Select a GPS station")
    display(gps_station)
else:
    print("No GPS stations found.")

In [ ]:
%matplotlib widget
scp_args = f"{mint_path}/velocity.h5 velocity --show-gps --ref-gps {gps_station.value} --gps-comp enu2los --gps-label --figsize 9 9"
with asfn.work_dir(mint_path):
    mintpy.view.main(scp_args.split())

# 5. Plotting a Motion Transect 

**Select the transect to plot**

In [ ]:
%matplotlib widget
image_file = f"{mint_path}/avgSpatialCoh.h5"
img = gdal.Open(image_file)
rasterstack = img.ReadAsArray()

line = asfn.LineSelector(rasterstack, 9, 9)

In [ ]:
amp = list(work_path.glob('*/*_amp_clip.tif'))[0]
amp = gdal.Open(str(amp))
geotrans = amp.GetGeoTransform()

def geolocation(x, y, geotrans):
    return [geotrans[0]+x*geotrans[1], geotrans[3]+y*geotrans[5]]

try:
    pnt_1 = geolocation(line.pnt1[0][0], line.pnt1[0][1], geotrans)
    pnt_2 = geolocation(line.pnt2[0][0], line.pnt2[0][1], geotrans)
    print(f"point 1: {pnt_1}")
    print(f"point 2: {pnt_2}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error may occur if a line was not selected.</text>'))

In [ ]:
%matplotlib inline
scp_args = f'{mint_path}/velocity.h5 --start-lalo {pnt_1[1]} {pnt_1[0]} --end-lalo {pnt_2[1]} {pnt_2[0]} --outfile x'
with asfn.work_dir(plot_path):
    mintpy.plot_transection.main(scp_args.split())

# 6. Plot the Cumulative Displacement Map and Point Displacement Time Series

- Use the `Time` bar below the Cumulative Displacement Map to view displacements for different time periods
- Click on the Cumulative Displacement Map to select points for displaying Point Displacement Time-Series

In [ ]:
%matplotlib widget
mintpy.tsview.main([ts_demErr, 
                    f'-d={mint_path}/inputs/geometryGeo.h5', 
                    f'-o={mint_path}/displacement_ts', 
                    f'--outfile={mint_path}/displacement_ts.pdf'])

# 7. Generate coherence, velocity, and total displacement Geotiffs

- **Determine the temporal range of the timeseries**
- **Define timeseries geotiff output filename**

In [ ]:
ifgramstack = inputs_path/"ifgramStack.h5"

with h5py.File(ifgramstack, "r") as f:
    dates = f["date"][()]
    d1 = dates[0][0].decode("utf-8") 
    d2 = dates[-1][-1].decode("utf-8")
    if correct_tropo:
        ts_tiff_name = f'{d1}_{d2}_timeseries_ERA5_demErr.tif'
    else:
        ts_tiff_name = f'{d1}_{d2}_timeseries_demErr.tif'
print(ts_tiff_name)

**Create a directory for the geotiffs**

In [ ]:
geotiff_path = mint_path/'GeoTiffs'
try:
    geotiff_path.mkdir()
except FileExistsError:
    pass

print(f"GeoTiffs directory: {geotiff_path}")

**Save the full displacement timeseries geotiff**

In [ ]:
ds = f'{d1}_{d2}'
!save_gdal.py $ts_demErr -d $ds --of GTIFF -o $geotiff_path/$ts_tiff_name

**Save the temporal coherence geotiff**

In [ ]:
!save_gdal.py $mint_path/temporalCoherence.h5 --of GTIFF -o $geotiff_path/TemporalCoherence.tif

**Save the average spatial coherence geotiff**

In [ ]:
!save_gdal.py $mint_path/avgSpatialCoh.h5 --of GTIFF -o $geotiff_path/avgSpatialCoh.tif

**Save the velocity geotiff**

In [ ]:
velocity_name = "velocity"
if correct_tropo:
    velocity_name = f'{velocity_name}ERA5'
vel_h5 = mint_path/f'{velocity_name}.h5'
vel_tiff = geotiff_path/f'{velocity_name}.tif'
!save_gdal.py $vel_h5 --of GTIFF -o $vel_tiff

# Reference material

- Mintpy reference: *Yunjun, Z., H. Fattahi, F. Amelung (2019), Small baseline InSAR time series analysis: unwrapping error correction and noise reduction, preprint doi:[10.31223/osf.io/9sz6m](https://eartharxiv.org/9sz6m/).*

- University of Miami online time-series viewer: https://insarmaps.miami.edu/

- Mintpy Github repository: https://github.com/insarlab/MintPy

*MintPy_Time_Series_From_Prepared_Data_Stack.ipynb - Version 0.2.0 - September 2022*

*Version Changes*

- *update pyaps3 config with CDS credentials in the notebook*
- *output full displacement timeseries geotiff*
- *output temporal and average spatial coherence geotiffs*
- *output velocity geotiff*